In [1]:
import cv2
import numpy as np


## grabs frame from camera one or two
def grabCam(camNum):
    ## cam indexes for camera number
    cam1 = 0
    cam2 = 1
    if(camNum==1):
        cam = cv2.VideoCapture(cam1)
        ret, frame = cam.read()    
        print(ret)
        cam.release()
        return frame
    if(camNum==2):
        cam = cv2.VideoCapture(cam2)
        ret, frame = cam.read()    
        print(ret)
        cam.release()
        return frame
 
## shows a frame
def showPic(fram):
    ## Show frame
    cv2.imshow("image",fram)
    k= cv2.waitKey(0)
    if k==27:
        cv2.destroyAllWindows()
    ## Deactivate Program
    cv2.destroyAllWindows()

## finds circle with color mask
def findCircle(frame):
    ## color mask
    lower = np.array([90,40,200])
    upper = np.array([140,100,255])
    frame = cv2.inRange(cv2.cvtColor(frame,cv2.COLOR_BGR2HSV),lower,upper)

    ## contour detection
    contours, _ = cv2.findContours(frame,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) >=1000:
            x,y,w,h = cv2.boundingRect(contour)
            if x<1100:
              return [x,y,w,h]

## finds square with color mask
def findSquare(frame):
    ## color mask
    lower = np.array([0,100,175])
    upper = np.array([12,200,255])
    frame = cv2.inRange(cv2.cvtColor(frame,cv2.COLOR_BGR2HSV),lower,upper)
    
    ## contour detection
    contours, _ = cv2.findContours(frame,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) >=1000:
            x,y,w,h = cv2.boundingRect(contour)
            if x<1100:
                return [x,y,w,h]

## finds triangle cross and star with single color mask
def findTriangleCrossStar(frame):
    ## initialize star and cross to -1 
    star = -1
    cross = -1
    maybeCross = -1
    maybeStar = -1
    triangle = -1 
    ## color mask
    lower = np.array([15,10,220])
    upper = np.array([60,150,255])
    frame = cv2.inRange(cv2.cvtColor(frame,cv2.COLOR_BGR2HSV),lower,upper)
    ## contour detection
    contours, _ = cv2.findContours(frame,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) >=1000:
            x,y,w,h = cv2.boundingRect(contour)
            if x<1100:
                epsilon = 0.02 * cv2.arcLength(contour,True)
                approx = cv2.approxPolyDP(contour,epsilon,True)
                print("1 ",len(approx))
                if len(approx) == 12 or len(approx)==13:
                    cross = [x,y,w,h]
                    continue
                if len(approx) == 11:
                    maybeCross = [x,y,w,h]
                    continue
                epsilon = 0.07 * cv2.arcLength(contour,True)
                approx = cv2.approxPolyDP(contour,epsilon,True)
                print("2 ",len(approx))
                if len(approx) == 5 or len(approx)==6:
                    star = [x,y,w,h]
                    continue
                if len(approx) == 4:
                    maybeStar = [x,y,w,h]
                    continue
                if len(approx) == 3:
                    triangle = [x,y,w,h]
    if cross ==-1:
        cross = maybeCross
    if star==-1 and cross!=maybeCross:
        star = maybeCross
    if star==-1:
        star = maybeStar
    if triangle == -1:
        triangle = maybeStar
    return triangle, cross, star

## returns list with [x,y,w,h] for circle, triangle, square, star, cross
def findBlocks(frame):
    circle = findCircle(frame)
    square = findSquare(frame)
    triangle, cross, star = findTriangleCrossStar(frame)
    return circle,triangle,square,star,cross

def findRobot(frame):
    ## sometimes robot area is smaller or splits into two contours so we will always pick the largest area
    ## initialize 
    robot = [0,0,0,0]
    area = 20
    ## lower and upper color bounds (hue,sat,val)
    ## hue maxes at 179 others 255
    lower = np.array([105,230,0])
    upper = np.array([120,255,255])
    
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    frame = cv2.inRange(frame,lower,upper)
    contours, _ = cv2.findContours(frame,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    ## look at each contour
    for contour in contours:
    ## to filter out noise make sure area over a certain ammount - try 800
        if cv2.contourArea(contour) >area:
            ## coordinates of top left contour
            x,y,w,h = cv2.boundingRect(contour)
            if x>400 and x<1500 and w*h>robot[2]*robot[3]:
                print("Robot",x,y,w,h)
                robot = [x,y,w,h]

    print("Final Robot",robot[0],robot[1],robot[2],robot[3])
    return robot

## shape is given by list [x,y,w,h]
def findMiddle(shape):
    x = shape[0]+(int(shape[2]/2))
    y = shape[1]+(int(shape[3]/2))
    return [x,y]

In [19]:
## "calibrate" or align board
## circle on left tape should middle roughly [968,821]
## square on right tape should middle roughly [1085,828]

## ideally y cords will be equal but it was coded with those parameters

frame = grabCam(1)
circle,triangle,square,star,cross = findBlocks(frame)
print("circle = ",findMiddle(circle),"  square = ",findMiddle(square))
showPic(frame)

True
circle =  [969, 819]   square =  [1090, 829]
